In [1]:
import pandas as pd
import numpy as np
import pulp

### MILP for Movie Scenes Scheduling
The Mixed Integer Linear Programming (MILP) approach is used to schedule the movie scenes. The obejective is to group scenes into days such that the total duration of scenes scheduled on each day does not exceed the maximum hours, while prioritizing scenes with the same location and actors.

Assumptions:
- The cost of each scene is a fixed number, i.e. the sequence of the scenes won’t affect the cost (including cost of props, actors, equipment, crews, etc.). However, switching locations between scenes will cause extra cost. 
- The durations of each scene are made up, randomly selected from a range from 1 to 6 hours with 30min interval.
- Each day has a maximum duration of 12 hours for shooting.
- Ignore the specific time requirement of the scene, such as dawn, dusk, etc.

Objective Function:
- w1 * pulp.lpSum(x[i, d] * d for i in scenes for d in days): Minimize the total number of days used.
- w2 * pulp.lpSum(y[location, d] for location in set(scene['location'] for scene in scenes.values()) for d in days): Minimize the number of different locations per day.
- w3 * pulp.lpSum(delta_location[location, d] for location in set(scene['location'] for scene in scenes.values()) for d in days if d > 1): Minimize the number of location switches between consecutive days.
- w4 * pulp.lpSum(delta_actors_between_scenes[i, j, d] for i in scenes for j in scenes for d in days if i != j): Minimize the number of actor switches between consecutive scenes within a day.
- w5 * pulp.lpSum(delta_actor[actor, d] for actor in actors for d in days if d > 1): Minimize the number of actor switches between consecutive days.

Constraints:
- Each scene is scheduled exactly once.
- The total duration of scenes in any day does not exceed 12 hours.
- Encourage scenes in the same location are assigned to the same day.
- Encourage scenes in the same location are assigned to the same day.
- Encourage consecutive days to have the same location.

Note:
The MILP algorithm is not very efficient, it takes hours to run on all scenes of the whole movie. Hence, here we only test on the first 20 scenes.

#### MILP Algorithm

In [2]:
def MILP(scenes, days):

    # Create the MILP problem
    prob = pulp.LpProblem("Scene_Scheduling_Problem", pulp.LpMinimize)

    # Define binary variables for scene-day assignment
    x = pulp.LpVariable.dicts("x", ((i, d) for i in scenes for d in days), cat='Binary')

    # Define binary variables for location-day assignment
    y = pulp.LpVariable.dicts("y", ((location, d) for location in set(scene['location'] for scene in scenes.values()) for d in days), cat='Binary')

    # Define binary variables for actor-day assignment
    actors = set(actor for scene in scenes.values() for actor in scene['actors'])
    z = pulp.LpVariable.dicts("z", ((actor, d) for actor in actors for d in days), cat='Binary')

    # Define variables for the order of scenes within each day
    order = pulp.LpVariable.dicts("order", ((i, j, d) for i in scenes for j in scenes for d in days if i != j), cat='Binary')

    # Define auxiliary variables for absolute differences
    delta_location = pulp.LpVariable.dicts("delta_location", ((location, d) for location in set(scene['location'] for scene in scenes.values()) for d in days if d > 1), lowBound=0, cat='Continuous')
    delta_actor = pulp.LpVariable.dicts("delta_actor", ((actor, d) for actor in actors for d in days if d > 1), lowBound=0, cat='Continuous')

    # Define binary variables for actor switches between scenes on the same day
    delta_actors_between_scenes = pulp.LpVariable.dicts("delta_actors_between_scenes", ((i, j, d) for i in scenes for j in scenes for d in days if i != j), cat='Binary')

    # Coefficients
    w1 = 100      # weight for minimizing total number of days used
    w2 = 1000    # weight for minimizing the number of different locations per day
    w3 = 10     # weight for minimizing the number of location switches between consecutive days
    w4 = 10    # weight for minimizing the number of actor switches between consecutive scenes on same day
    w5 = 1    # weight for minimizing the number of actor switches between consecutive days

    # Objective function
    prob += (
        w1 * pulp.lpSum(x[i, d] * d for i in scenes for d in days) +
        w2 * pulp.lpSum(y[location, d] for location in set(scene['location'] for scene in scenes.values()) for d in days) +
        w3 * pulp.lpSum(delta_location[location, d] for location in set(scene['location'] for scene in scenes.values()) for d in days if d > 1) +
        w4 * pulp.lpSum(delta_actors_between_scenes[i, j, d] for i in scenes for j in scenes for d in days if i != j) +
        w5 * pulp.lpSum(delta_actor[actor, d] for actor in actors for d in days if d > 1)
    )

    # Constraints: each scene is scheduled exactly once
    for i in scenes:
        prob += pulp.lpSum(x[i, d] for d in days) == 1

    # Constraints: the total duration of scenes in any day does not exceed 12 hours
    for d in days:
        prob += pulp.lpSum(x[i, d] * scenes[i]['duration'] for i in scenes) <= 12

    # Ensure scenes in the same location are assigned to the same day
    for location in set(scene['location'] for scene in scenes.values()):
        for d in days:
            scenes_in_location = [i for i, scene in scenes.items() if scene['location'] == location]
            if scenes_in_location:
                prob += pulp.lpSum(x[i, d] for i in scenes_in_location) <= 12 * y[location, d]  # If y[location, d] is 1, allow up to 12 hours of scenes

    # Ensure that scenes with the same location are grouped in consecutive days
    for d in days[:-1]:
        for location in set(scene['location'] for scene in scenes.values()):
            prob += y[location, d + 1] >= y[location, d] - 1  # Encourage consecutive days to have the same location

    # Absolute difference constraints for locations
    for location in set(scene['location'] for scene in scenes.values()):
        for d in days:
            if d > 1:
                prob += delta_location[location, d] >= y[location, d] - y[location, d - 1]
                prob += delta_location[location, d] >= y[location, d - 1] - y[location, d]

    # Ensure scenes with the same actors are assigned to the same day
    for actor in actors:
        for d in days:
            scenes_with_actor = [i for i, scene in scenes.items() if actor in scene['actors']]
            if scenes_with_actor:
                prob += pulp.lpSum(x[i, d] for i in scenes_with_actor) <= 12 * z[actor, d]  # If z[actor, d] is 1, allow up to 12 hours of scenes

    # Absolute difference constraints for actors
    for actor in actors:
        for d in days:
            if d > 1:
                prob += delta_actor[actor, d] >= z[actor, d] - z[actor, d - 1]
                prob += delta_actor[actor, d] >= z[actor, d - 1] - z[actor, d]

    # Add constraints to minimize actor switches between scenes within a day
    for d in days:
        for i in scenes:
            for j in scenes:
                if i != j:
                    common_actors = set(scenes[i]['actors']).intersection(set(scenes[j]['actors']))
                    prob += delta_actors_between_scenes[i, j, d] >= x[i, d] + x[j, d] - 1 - (len(common_actors) > 0)  # If there are common actors, delta should be 0

    # Solve the problem
    prob.solve()

    # Print results
    schedule = {}
    for i in scenes:
        for d in days:
            if pulp.value(x[i, d]) == 1:
                if d not in schedule:
                    schedule[d] = []
                schedule[d].append(i)

    # Sort the schedule by days
    sorted_schedule = sorted(schedule.items())

    return sorted_schedule

#### Test on a Small Dataset

In [3]:
# Example data
scenes = {
    1: {'duration': 5, 'time': 'day', 'actors': ['A', 'B'], 'location': 'studio'},
    2: {'duration': 2, 'time': 'day', 'actors': ['D'], 'location': 'park'},
    3: {'duration': 3.5, 'time': 'day', 'actors': ['A','B'], 'location': 'studio'},
    4: {'duration': 3.5, 'time': 'day', 'actors': ['A', 'B'], 'location': 'park'},
    5: {'duration': 2, 'time': 'day', 'actors': ['A'], 'location': 'park'},
    6: {'duration': 6, 'time': 'day', 'actors': ['A','C'], 'location': 'studio'},
    7: {'duration': 2.5, 'time': 'day', 'actors': ['C'], 'location': 'studio'}
}

# Define days
# Assume we have up to 10 days available
days = range(1, 11)

# Call the MILP function
sorted_schedule = MILP(scenes, days)

print("Day schedule:")
for d, scenes in sorted_schedule:
    print(f"Day {d}: {scenes}")

Day schedule:
Day 1: [2, 4, 5]
Day 2: [3, 6, 7]
Day 3: [1]


#### Test on Movie Scenes

In [4]:
df = pd.read_csv('scenes_metadata.csv')
df = df[['scene_id', 'location', 'person']]
#df.head()

In [5]:
# synthesize time duration of scenes
min_val = 1
max_val = 6
interval = 0.5

# Set the random seed
np.random.seed(42)

# Generate random numbers
random_numbers = np.random.choice(np.arange(min_val, max_val + interval, interval), size=len(df))

# Add the time duration column
df['duration'] = random_numbers

In [6]:
scenes = {}
for i in range(len(df)):
    scene = {}
    scene['location'] = df['location'][i][1:]
    scene['actors'] = df['person'][0][:-1].split(',')
    scene['duration'] = df['duration'][i]
    scenes[i] = scene

In [7]:
# Scenes Data
scenes = dict(list(scenes.items())[:20])

# Define days
# Assume we have up to 20 days
max_possible_days = 20
days = range(1, max_possible_days + 1)

# Call the MILP function
sorted_schedule = MILP(scenes, days)

print("Day schedule:")
for d, scenes in sorted_schedule:
    print(f"Day {d}: {scenes}")

Day schedule:
Day 1: [1, 4, 7, 13, 16]
Day 2: [5, 17, 18, 19]
Day 3: [0, 8, 12]
Day 4: [2, 10]
Day 5: [3, 6]
Day 6: [9, 15]
Day 7: [11, 14]


In [8]:
df.head(20)

,scene_id,location,person,duration
0,0,"OFFICE, HIGH RISE","DOPEY,HAPPY,",4.0
1,1,HIGH-RISE,NaN,2.5
2,2,DOWNTOWN GOTHAM,"GRUMPY,CHUCKLES,JOKER,BOZO,",6.0
3,3,"ROOFTOP, BANK","DOPEY,HAPPY,",4.5
4,4,BANK,"GRUMPY,CHUCKLES,BOZO,",3.0
5,5,BANK,"GRUMPY,BOZO,",4.0
6,6,ROOFTOP,"DOPEY,HAPPY,",5.5
7,7,"STAIRWELL, BANK",NaN,2.0
8,8,"VAULT ROOM, BANK",NaN,4.0
9,9,"LOBBY, BANK","BOZO,GRUMPY,",6.0
